<a href="https://colab.research.google.com/github/hyeonjenny/ai-security/blob/master/First%20assignment/%EC%9A%B0%ED%98%84%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# nn 패키지를 사용하여 신경망 정의를 한다.
# nn 패키지는 모델을 정의하고 미분하는데 autograd를 사용한다.

import torch # torch 모듈을 가져온다.
import torch.nn as nn # torch.nn을 nn 모듈로 가져온다. 
import torch.nn.functional as F #torch.nn.functional을 F모듈로 가져온다.

#nn.Module은 신경망(Neural Network) 모듈로
#파라미터를 GPU로 옮기거나, 전송, 로드 등 파라미터를 캡슐화하는 편리한 방법이다.
class Net(nn.Module): #nn.Module 을 통해 Net 클래스를 정의한다.

    def __init__(self):#
        super(Net, self).__init__()
       # 입력 이미지: 1개
        # 출력 체널: 6개
        # 커널 크기: 5X5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 선형 모델: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        #x를 매개변수로 forward 함수를 정의해준다.
        #forward함수에서는 모든 Tensor연산을 사용할 수 있다.
    def forward(self, x):
        # (2, 2) 윈도우로 MaxPooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # size가 정사각형인 경우, (2,2) 대신 2로 하나의 숫자를 지정하여 쓰기도 한다.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # view는 텐서의 크기를 변형하는 함수이다.
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x #x 초기화
        
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size: # size라는 목록의 각각의 항목 s에 대하여
            num_features *= s #num features = num features * s 이다.
        return num_features #초기화


net = Net()
print(net)

In [0]:
#모든 학습가능한 파라미터는 net.parameters()에 저장돼있다.

params = list(net.parameters()) #파라미터에 대한 리스트 생성.
print(len(params)) #파라미터 목록에 몇 개가 들어있는지를 출력해준다.
print(params[0].size())  # conv1의 가중치 크기

In [0]:
#forward함수의 입력은 autograd.Variable(자동미분변수)이므로 ouput도 autograd.Variable(자동미분변수)이다.
#이 네트워크의 예상 input size는 32X32이다. 
#이 네트워크에  MNIST 데이터셋을 사용하기 위해서는 크기를 32x32로 변경해야 한다.
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)
# 랜덤함수로 토치를 만들고 net에 통과시켰다.

In [0]:
#모든 파라미터의 그라디언트 버퍼를 0으로 설정하고
net.zero_grad()
#랜덤으로 그라디언트를 역전파한다.
out.backward(torch.randn(1, 10))

In [0]:
#이제부터 nn패키지에 존재하는 여러 손실 함수들 중 nn.MSELoss를 사용한다.
output = net(input)
target = torch.randn(10)  # 예제로 사용할 더미 타켓
target = target.view(1, -1) # 동일한 형태의 더미 타켓
#nn.MSELoss는 오차 함수이다.
#이 함수는 입력과 타켓의 평균 제곱 오차를 계산한다.
criterion = nn.MSELoss() 


loss = criterion(output, target)
print(loss)

In [0]:
print(loss.grad_fn)  # 출력함수를 통해 grand텐서 출력 #MSELoss
print(loss.grad_fn.next_functions[0][0])  # 출력함수를 통해 grand텐서 출력# Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # 출력함수를 통해 grand텐서 출력 # ReLU

In [0]:
net.zero_grad()     # 모든 파라미터의 그라디언트 버퍼를 0으로 한다.

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad) #출력함수를 통해 conv1 바이어스 그라디언트 값을 출력한다.

loss.backward() #loss.backward()를 호출하면 전체 그래프는 손실에 대해 미분이 계산된다.
#오차를 역전파하기 위해 loss.backward()를 호출하여 존재하는 그라디언트를 초기화 해야한다.
#초기화하지 않으면 그라디언트에 그라디언트가 누적되어 저장된다.


print('conv1.bias.grad after backward') #출력함수를 통해 backwardgks conv1의 바이어스 그라디언트 값을 출력한다.
print(net.conv1.bias.grad)

In [0]:
import torch.optim as optim # torch.optim 을 optim 모듈로 가져온다. 

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # 그라디언트 버퍼를 0으로 초기화해준다.
output = net(input)
loss = criterion(output, target)
loss.backward() # loss.backward함수로 그라디언트를 초기화해준다.
optimizer.step()    # Does the update

#tutorial 출처 :
#https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#

#참고자료:
#http://taewan.kim/trans/pytorch/tutorial/blits/03_neural_networks/
#https://blog.naver.com/PostView.nhn?blogId=keum_zz6&logNo=221319428565&parentCategoryNo=&categoryNo=15&viewDate=&isShowPopularPosts=true&from=search
#https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/former_torchies/nn_tutorial.html